In [1]:
using StatisticalRethinking, Distributed, JLD
using Mamba

# Data
line = Dict{Symbol, Any}()

howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

Use only adults

In [2]:
df2 = filter(row -> row[:age] >= 18, df);
line[:x] = convert(Array{Float64,1}, df2[:weight]);
line[:y] = convert(Array{Float64,1}, df2[:height]);
line[:xmat] = convert(Array{Float64,2}, [ones(length(line[:x])) line[:x]])

# Model Specification
model = Model(
  y = Stochastic(1,
    (xmat, beta, s2) -> MvNormal(xmat * beta, sqrt(s2)),
    false
  ),
  beta = Stochastic(1, () -> MvNormal([178, 0], [sqrt(10000), sqrt(100)])),
  s2 = Stochastic(() -> Uniform(0, 50))
)

# Initial Values
inits = [
  Dict{Symbol, Any}(
    :y => line[:y],
    :beta => [rand(Normal(178, 100)), rand(Normal(0, 10))],
    :s2 => rand(Uniform(0, 50))
  )
  for i in 1:3
]

# Tuning Parameters
scale1 = [0.5, 0.25]
summary1 = identity
eps1 = 0.5

scale2 = 0.5
summary2 = x -> [mean(x); sqrt(var(x))]
eps2 = 0.1

# Define sampling scheme

scheme = [
  Mamba.NUTS([:beta]),
  Mamba.Slice([:s2], 10)
]

setsamplers!(model, scheme)

# MCMC Simulation

sim = mcmc(model, line, inits, 10000, burnin=1000, chains=3)

MCMC Simulation of 10000 Iterations x 3 Chains...

Chain 1:   0% [0:12:41 of 0:12:42 remaining]
Chain 1:  10% [0:00:58 of 0:01:05 remaining]
Chain 1:  20% [0:00:55 of 0:01:09 remaining]
Chain 1:  30% [0:00:49 of 0:01:10 remaining]
Chain 1:  40% [0:00:41 of 0:01:08 remaining]
Chain 1:  50% [0:00:35 of 0:01:10 remaining]
Chain 1:  60% [0:00:29 of 0:01:14 remaining]
Chain 1:  70% [0:00:22 of 0:01:13 remaining]
Chain 1:  80% [0:00:15 of 0:01:13 remaining]
Chain 1:  90% [0:00:07 of 0:01:12 remaining]
Chain 1: 100% [0:00:00 of 0:01:13 remaining]

Chain 2:   0% [0:00:02 of 0:00:02 remaining]
Chain 2:  10% [0:00:48 of 0:00:53 remaining]
Chain 2:  20% [0:00:54 of 0:01:08 remaining]
Chain 2:  30% [0:00:48 of 0:01:09 remaining]
Chain 2:  40% [0:00:42 of 0:01:10 remaining]
Chain 2:  50% [0:00:35 of 0:01:10 remaining]
Chain 2:  60% [0:00:28 of 0:01:10 remaining]
Chain 2:  70% [0:00:21 of 0:01:10 remaining]
Chain 2:  80% [0:00:14 of 0:01:10 remaining]
Chain 2:  90% [0:00:07 of 0:01:10 remaining]
Cha

Object of type "Mamba.ModelChains"

Iterations = 1001:10000
Thinning interval = 1
Chains = 1,2,3
Samples per chain = 9000

[21.489 116.607 0.846279; 28.8521 116.607 0.846279; … ; 25.7151 112.194 0.939912; 30.532 113.487 0.909322]

[25.7584 115.856 0.868934; 23.9949 115.253 0.884889; … ; 26.0662 114.037 0.899618; 25.9261 111.586 0.954614]

[26.4942 110.947 0.972804; 25.1066 110.947 0.963033; … ; 24.735 110.452 0.969289; 28.0403 114.933 0.877591]

Show draws summary

In [3]:
describe(sim)

Iterations = 1001:10000
Thinning interval = 1
Chains = 1,2,3
Samples per chain = 9000

Empirical Posterior Estimates:
            Mean         SD        Naive SE       MCSE        ESS   
     s2  26.1670729 1.995706131 0.01214548074 0.0197657876 9000.0000
beta[1] 113.9317509 1.902244407 0.01157669080 0.0351756191 2924.4832
beta[2]   0.9038741 0.041865556 0.00025478566 0.0007675938 2974.7525

Quantiles:
            2.5%       25.0%       50.0%        75.0%        97.5%   
     s2  22.5651124  24.7723696  26.0486741  27.45604979  30.34173672
beta[1] 110.2276855 112.6434445 113.9159590 115.21148729 117.73867915
beta[2]   0.8205039   0.8754851   0.9039817   0.93273913   0.98498503



End of `clip_38.0m.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*